In [ ]:
import ipychart as ipyc

In [ ]:
chart = ipyc.BarChart()
chart.set_data([10, 20, 30, 40, 50, 80])
chart

In [55]:
import ipywidgets as widgets
from traitlets import Unicode, default, List

class DevChart(widgets.DOMWidget):
    """An example widget."""
    _view_name = Unicode('DevChartView').tag(sync=True)
    _view_module = Unicode('dev').tag(sync=True)
    
    _label = List([]).tag(sync=True)
    _dataset = List([]).tag(sync=True)
    
    _type = Unicode().tag(sync=True)
    
    def __init__(self, data, kind):
        
        super().__init__()
        
        assert kind in ['line', 'bar', 'horizontalBar', 'radar', 'doughnut', 'polarArea',
                        'bubble'], 'Type must be one of : line, bar, radar, doughnut, polarArea, bubble'
        assert isinstance(data, (list, dict)), 'Please enter data as dict of list'

        if isinstance(data, list):
            self._data = data
            self._label = [str(i + 1) for i in range(len(self._data))]

        else:
            assert 'data' in data, 'Please input data using the key "data" in the dict'
            if 'labels' in data:
                assert len(data['labels']) == len(data['data']), 'Please enter one label for each datapoint'
                
            self._data = data['data']
            self._label = data['labels'] if 'labels' in data else [str(i + 1) for i in range(len(self._data))]
        
        print(self._data)
        print(self._label)
        self._dataset = self._create_dataset(self._data, self._label)
        print(self._dataset)
        self._type = kind
        
    @staticmethod
    def _create_dataset(data, label):
        
        dataset = list() 
        # todo: handle case of radar, doughnut and polar area
        for index, value in enumerate(data):
            dataset.append({'label': label[index], 'data': [value]})
        
        return dataset
            

    @default('layout')
    def _default_layout(self):
        return widgets.Layout(height='auto', align_self='stretch')
    

In [60]:
%%javascript
require.undef('dev');


define('dev', ["@jupyter-widgets/base", "./js/node_modules/chart.js/dist/Chart.min.js"], function(widgets) {
    var DevChartView = widgets.DOMWidgetView.extend({
    render: function() {
        
        var dataset = this.model.get("_dataset");
        var label = this.model.get("_label");
        var type = this.model.get("_type");
        
        console.log(dataset);
        console.log(label);
        
        let xaxis_display = true;
        let yaxis_display = true;
        if (['radar', 'doughnut', 'polarArea'].indexOf(type) >= 0) {
            xaxis_display = false;
            yaxis_display = false;
        }

        
        let canvas = document.createElement("canvas");
        var ctx = canvas.getContext('2d');
        
        new Chart(ctx, {
            type: type,
            data: {
                labels: label,
                datasets: dataset
            },
            options: {
                legend: {
                    display: true
                },
                tooltips: {
                    callbacks: {
                       label: function(tooltipItem) {
                              return tooltipItem.yLabel;
                       }
                    }
                },
                scales: {
                    xAxes: [{
                        display: xaxis_display,
                        ticks: {
                            display: xaxis_display
                        }
                    }],
                    yAxes: [{
                        display: yaxis_display,
                        ticks: {
                            beginAtZero: true,
                            display: yaxis_display
                        }
                    }]
                }
            }
        });
        this.el.appendChild(canvas);
        }
    });
    
    return {
        DevChartView : DevChartView
    };
});

<IPython.core.display.Javascript object>

In [61]:
data = {'data': [12, 19, 11, 5, 14, 17], 'labels': ['test', 'test1', 'test2', 'test3', 'test4', 'test5']}

test_wd = DevChart(data, 'bar')
test_wd

[12, 19, 11, 5, 14, 17]
['test', 'test1', 'test2', 'test3', 'test4', 'test5']
[{'label': 'test', 'data': [12]}, {'label': 'test1', 'data': [19]}, {'label': 'test2', 'data': [11]}, {'label': 'test3', 'data': [5]}, {'label': 'test4', 'data': [14]}, {'label': 'test5', 'data': [17]}]


DevChart(layout=Layout(align_self='stretch', height='auto'))